In [1]:
import glob, os, sys, toml, pickle, time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from os import path

from sklearn.preprocessing import OneHotEncoder
from multiprocessing import Pool, cpu_count
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor



path_MOASMO = '/glade/u/home/guoqiang/CTSM_repos/ctsm_optz/MO-ASMO/src/'
sys.path.append(path_MOASMO)
import gp
import NSGA2


import os, sys, subprocess, time, toml

sys.path.append("/glade/u/home/mozhgana/mywork/model_calibration/src/moasmo_test/")
sys.path.append("/glade/u/home/mozhgana/mywork/model_calibration/src/moasmo_test/allbasin_emulator")
from MOASMO_parameter_allbasin_emulator import *
from multiprocessing import Pool


sys.path.append("/glade/u/home/guoqiang/CTSM_repos/ctsm_optz/MO-ASMO/src")
# import NSGA2


from MOASMO_parameters import *

import torch
import torch.nn as nn
import torch.optim as optim


import torch
import torch.nn as nn
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
infile_basin_info = '/glade/work/guoqiang/CTSM_CAMELS/data_mesh_surf/HillslopeHydrology/CAMELS_level1_basin_info.csv'
infile_param_info = '/glade/u/home/mozhgana/mywork/model_calibration/src/moasmo_test/param_file_tpl.csv'
infile_attr_foruse = '/glade/u/home/guoqiang/CTSM_repos/CTSM_calibration/data/camels_attributes_table_TrainModel.csv'
inpath_moasmo = '/glade/campaign/cgd/tss/people/mozhgana/projects/SUMMA_Calib'

CV=1
ncpus = 1
iterend = 1
outpathname = 'LSE_spaceCV_PredictParam_4X_ann'
suffix = f'LSEspaceCV_ann_{CV}'
objfunc = 'normKGE'
numruns = 100


In [3]:
CV=1
outpath = '/glade/campaign/cgd/tss/people/mozhgana/projects/SUMMA_Calib/LSE_spaceCV_PredictParam'
# divide into train/test index
outfile = f'{outpath}/train_test_CV_indices.npz'
if os.path.isfile(outfile):
    dtmp = np.load(outfile, allow_pickle=True)
    train_indices, test_indices = dtmp['train_indices'], dtmp['test_indices']

train_index= train_indices[CV-1]
test_index= test_indices[CV-1]

In [4]:
suffix_defa_source = 'LSEnormKGE'

outpath = f"{inpath_moasmo}/{outpathname}"
os.makedirs(outpath, exist_ok=True)

# Load data: same for all iterations
df_basin_info = pd.read_csv(infile_basin_info)
df_basin_info.index = np.arange(len(df_basin_info))
all_index = np.arange(len(df_basin_info))

test_index = np.setdiff1d(all_index, train_index)

# information for all basins
df_param_info = pd.read_csv(infile_param_info)

file_defa_param = f'{outpath}/camels_summa_defa_param_train_{suffix}.csv'
df_param_defa_train = read_allbasin_defa_params(infile_param_info, file_defa_param, Basin_list, train_index)

file_defa_param = f'{outpath}/camels_summa_defa_param_test_{suffix}.csv'
df_param_defa_test = read_allbasin_defa_params(infile_param_info, file_defa_param, Basin_list, test_index)

file_param_lb = f'{outpath}/camels_summa_all_param_lb_train_{suffix}.gz'
file_param_ub = f'{outpath}/camels_summa_all_param_ub_train_{suffix}.gz'

df_param_lb_train, df_param_ub_train = load_basin_param_bounds(inpath_moasmo, df_param_defa_train, file_param_lb, file_param_ub)

file_param_lb = f'{outpath}/camels_summa_all_param_lb_test_{suffix}.gz'
file_param_ub = f'{outpath}/camels_summa_all_param_ub_test_{suffix}.gz'
df_param_lb_test, df_param_ub_test = load_basin_param_bounds(inpath_moasmo, df_param_defa_test, file_param_lb, file_param_ub)


file_camels_attribute = f'{outpath}/camels_basin_attribute_train_{suffix}.pkl'
df_att_train = read_camels_attributes(infile_basin_info, file_camels_attribute, train_index)
file_camels_attribute = f'{outpath}/camels_basin_attribute_test_{suffix}.pkl'
df_att_test = read_camels_attributes(infile_basin_info, file_camels_attribute, test_index)

df_att_foruse = pd.read_csv(infile_attr_foruse)
useattrs = list(df_att_foruse[df_att_foruse['att_Xie2021'].values]['Attribute_text'].values)
print("The number of attributes used:", len(useattrs))
print(useattrs)


suffixtest = suffix+'test'

# Load data: outputs from each iteration from training basins
for iter in range(0, iterend):
    file_all_param = f'{outpath}/camels_summa_all_param_train_{suffix}_iter{iter}.gz'
    file_all_metric = f'{outpath}/camels_summa_all_metric_train_{suffix}_iter{iter}.gz'
    file_all_basinid = f'{outpath}/camels_summa_all_basinid_train_{suffix}_iter{iter}.gz'

    file_all_param_test = f'{outpath}/camels_summa_all_param_test_{suffix}_iter{iter}.gz'
    file_all_metric_test = f'{outpath}/camels_summa_all_metric_test_{suffix}_iter{iter}.gz'
    file_all_basinid_test = f'{outpath}/camels_summa_all_basinid_test_{suffix}_iter{iter}.gz'

    if iter == 0:

        df_param_i, df_metric_i, df_basinid_i = load_all_basin_params_metrics(inpath_moasmo, infile_param_info, df_param_defa_train,
                                                                              df_basin_info, iter, file_all_param,
                                                                              file_all_metric, file_all_basinid,
                                                                              train_index, suffix_defa_source)
    
        df_param_i_test, df_metric_i_test, df_basinid_i_test = load_all_basin_params_metrics(inpath_moasmo, infile_param_info, df_param_defa_test,
                                                                              df_basin_info, iter, file_all_param_test,
                                                                              file_all_metric_test, file_all_basinid_test,
                                                                              test_index, suffix_defa_source)
    else:
        df_param_i, df_metric_i, df_basinid_i = load_all_basin_params_metrics(inpath_moasmo, infile_param_info, df_param_defa_train,
                                                                              df_basin_info, iter, file_all_param,
                                                                              file_all_metric, file_all_basinid,
                                                                              train_index, suffix)
    
        df_param_i_test, df_metric_i_test, df_basinid_i_test = load_all_basin_params_metrics(inpath_moasmo, infile_param_info, df_param_defa_test,
                                                                              df_basin_info, iter, file_all_param_test,
                                                                              file_all_metric_test, file_all_basinid_test,
                                                                              test_index, suffixtest)

    df_basinid_i['iter'] = iter
    df_basinid_i_test['iter'] = iter

    if iter == 0:
        df_param = df_param_i
        df_metric = df_metric_i
        df_basinid = df_basinid_i
        
        df_param_test = df_param_i_test
        df_metric_test = df_metric_i_test
        df_basinid_test = df_basinid_i_test
    else:
        df_param = pd.concat([df_param, df_param_i])
        df_metric = pd.concat([df_metric, df_metric_i])
        df_basinid = pd.concat([df_basinid, df_basinid_i])
        
        df_param_test = pd.concat([df_param_test, df_param_i_test])
        df_metric_test = pd.concat([df_metric_test, df_metric_i_test])        
        df_basinid_test = pd.concat([df_basinid_test, df_basinid_i_test])

df_param = df_param.apply(pd.to_numeric, errors='coerce')
df_param_test = df_param_test.apply(pd.to_numeric, errors='coerce')


df_param.index = np.arange(len(df_param))
df_metric.index = np.arange(len(df_metric))
df_basinid.index = np.arange(len(df_basinid))

df_param_test.index = np.arange(len(df_param_test))
df_metric_test.index = np.arange(len(df_metric_test))
df_basinid_test.index = np.arange(len(df_basinid_test))


index = np.isnan(np.sum(df_metric.values, axis=1) + np.sum(df_param.values, axis=1))
df_param = df_param[~index]
df_metric = df_metric[~index]
df_basinid = df_basinid[~index]

index_test = np.isnan(np.sum(df_metric_test.values, axis=1) + np.sum(df_param_test.values, axis=1))
df_param_test = df_param_test[~index_test]
df_metric_test = df_metric_test[~index_test]
df_basinid_test = df_basinid_test[~index_test]


df_param.index = np.arange(len(df_param))
df_metric.index = np.arange(len(df_metric))
df_basinid.index = np.arange(len(df_basinid))

df_param_test.index = np.arange(len(df_param_test))
df_metric_test.index = np.arange(len(df_metric_test))
df_basinid_test.index = np.arange(len(df_basinid_test))


print('Number of nan samples:', np.sum(index))
print("Number of original parameter sets:", len(index))
print("Number of final parameter sets:", len(df_param))


# One-hot encoding for categorical attributes
df_att = pd.concat([df_att_train, df_att_test])
df_att.index = np.arange(len(df_att))
df_att_use = df_att[useattrs + ["hru_id"]]

for att in useattrs:
    if df_att_use[att].dtype == "object":
        print('Convert', att, 'to one-hot encoding')
        enc = OneHotEncoder(sparse_output=False)
        enc.fit(df_att_use[[att]])
        encnames = [att + "_" + str(i) for i in range(len(enc.categories_[0]))]
        print('New columns:', encnames)
        df_enc = pd.DataFrame(enc.transform(df_att_use[[att]]), columns=encnames)
        df_att_use = pd.concat([df_att_use, df_enc], axis=1)
        df_att_use = df_att_use.drop([att], axis=1)

df_att_use_train = df_att_use[:len(df_att_train)]
df_att_use_test = df_att_use[len(df_att_train):]
df_att_use_train.index = np.arange(len(df_att_use_train))
df_att_use_test.index = np.arange(len(df_att_use_test))

useattrs = list(df_att_use_train.columns)
useattrs.remove('hru_id')

useattrs_test = list(df_att_use_test.columns)
useattrs_test.remove('hru_id')

# Prepare model input and output
df_input = df_param.copy()
df_input["hru_id"] = df_basinid["basin_name"]
df_input = df_input.merge(df_att_use_train[useattrs + ["hru_id"]], on="hru_id", how="left")
df_input = df_input.drop(["hru_id"], axis=1)

inputnames = list(df_param.columns) + useattrs
x_all = df_input[inputnames].values.copy()
print("Input shape:", x_all.shape)



File exists: /glade/campaign/cgd/tss/people/mozhgana/projects/SUMMA_Calib/LSE_spaceCV_PredictParam_4X_ann/camels_basin_attribute_train_LSEspaceCV_ann_1.pkl
File exists: /glade/campaign/cgd/tss/people/mozhgana/projects/SUMMA_Calib/LSE_spaceCV_PredictParam_4X_ann/camels_basin_attribute_test_LSEspaceCV_ann_1.pkl
The number of attributes used: 27
['mean_elev', 'mean_slope', 'area_gauges2', 'p_mean', 'pet_mean', 'aridity', 'p_seasonality', 'frac_snow', 'high_prec_freq', 'high_prec_dur', 'low_prec_freq', 'low_prec_dur', 'frac_forest', 'lai_max', 'lai_diff', 'dom_land_cover', 'dom_land_cover_frac', 'soil_depth_pelletier', 'soil_depth_statsgo', 'soil_porosity', 'soil_conductivity', 'max_water_content', 'sand_frac', 'silt_frac', 'clay_frac', 'carbonate_rocks_frac', 'geol_permeability']
Number of nan samples: 406
Number of original parameter sets: 200400
Number of final parameter sets: 199994
Convert dom_land_cover to one-hot encoding
New columns: ['dom_land_cover_0', 'dom_land_cover_1', 'dom_la

In [5]:
    # Normalize the features
    x_train_mean = np.mean(x_all, axis=0)
    x_train_std = np.std(x_all, axis=0)    
    x_all_scaled = (x_all - x_train_mean) / x_train_std

In [7]:
    if objfunc == 'normKGE':
        print('Use normalized KGE as output')
        df_output = df_metric.copy()
        y_all = df_output[["kge"]].values.copy()
        y_all = y_all / (2 - y_all)  # Normalize KGE

        # File path to save or load the model
        outfile = f'/glade/derecho/scratch/guoqiang/MLP_emulator_for_iter{iterend}_{suffix}.pkl'
        
        # Check if the model file exists
        if os.path.isfile(outfile):
            # Load the existing model
            with open(outfile, 'rb') as file:
                em_model = pickle.load(file)
        else:
            # Model configuration for MLP
            modelconfig = {
                'hidden_layer_sizes': (100, 100),
                'max_iter': 1000,
                'alpha': 0.001,
                'random_state': 42,
                'early_stopping': True,
                'validation_fraction': 0.1,  # 10% internal validation for early stopping
                'n_iter_no_change': 10
            }
        
            # Initialize and train the model
            em_model = MLPRegressor(**modelconfig)
            em_model.fit(x_all_scaled, y_all)
        
            # Save the trained model
            with open(outfile, 'wb') as file:
                pickle.dump(em_model, file)

Use normalized KGE as output


In [16]:
            modelconfig = {'n_estimators': 200, 'random_state': 42, 'max_depth': 40}
            em_model = RandomForestRegressor(**modelconfig, n_jobs=ncpus)
            em_model.fit(x_all_scaled, y_all)

/glade/work/guoqiang/conda-envs/PytorchEnv/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(max_depth=40, n_estimators=200, n_jobs=1, random_state=42)

In [8]:
y_all_predict = em_model.predict(x_all_scaled)
np.sqrt(np.mean( (np.squeeze(y_all_predict)-np.squeeze(y_all))**2))

np.float64(0.04822202284487172)

In [8]:
    # normdict = {'method': 'z-score',
    #             'mean': x_train_mean,
    #             'std': x_train_std,
    #            }
    # param_names = df_param_info['Parameter'].values
    # parallel_process_basins_norm(df_basinid, df_param_lb_train, df_param_ub_train,
    #                         x_all, df_input, y_all,
    #                         param_names, inputnames, em_model, inpath_moasmo, ncpus, numruns, iterend, train_index, suffix, normdict)


In [9]:
    #### predict parameter in unseen basins
    suffixtest = suffix+'test'
    numruns_test = 1 # can be smaller
    if objfunc == 'normKGE':
        num_objfunc=1
    else:
        sys.exit('Not tested objfunc')

    df_att_use_test2 = df_att_use_test.drop(['hru_id'], axis=1)

    xlb_mean_test = np.nan * np.zeros([len(df_param_lb_test), len(inputnames)])
    xub_mean_test = np.nan * np.zeros([len(df_param_ub_test), len(inputnames)])   
    for i in range(len(df_param_lb_test)):
        param_lb_mean = df_param_lb_test.values[i, :]
        param_ub_mean = df_param_ub_test.values[i, :]
        attrvalues = df_att_use_test2.values[i,:]
        xlb_mean_test[i,:] = np.hstack([param_lb_mean, attrvalues])
        xub_mean_test[i,:] = np.hstack([param_ub_mean, attrvalues])

    xlb_mean_test_scaled = (xlb_mean_test - x_train_mean) / x_train_std
    xub_mean_test_scaled = (xub_mean_test - x_train_mean) / x_train_std
    normdict = {'method': 'z-score',
                'mean': x_train_mean,
                'std': x_train_std,
               }
    
    # parallel_process_basins_predictunseen_norm(xlb_mean_test_scaled, xub_mean_test_scaled, param_names, em_model, inpath_moasmo, ncpus, numruns_test, iterend, test_index, suffixtest, num_objfunc, normdict)


In [10]:
    times = 100
    param_names = df_param_info['Parameter'].values
    df_basinid, xlb_mean_all, xub_mean_all, param_names, em_model, inpath_moasmo, ncpus, numruns, iterend, basin_index, suffix, num_objfunc, normdict = df_basinid_test, xlb_mean_test_scaled, xub_mean_test_scaled, param_names, em_model, inpath_moasmo, ncpus, numruns_test, iterend, test_index, suffixtest, num_objfunc, normdict
    args = [(basin_index[tarbasin_id], tarbasin_id, df_basinid, em_model, xlb_mean_all[tarbasin_id, :], xub_mean_all[tarbasin_id, :], 
                 param_names, inpath_moasmo, numruns, iterend, suffix, num_objfunc, normdict, times) 
                for tarbasin_id in range(len(basin_index))]

    tarbasin, tarbasin_id, df_basinid, em_model, xlb_mean, xub_mean, param_names, inpath_moasmo, numruns, iterend, suffix, num_objfunc, normdict, times = args[0]

    basin_id= str(int(df_basinid.loc[df_basinid['basin_id'] == tarbasin, 'basin_name'].values[0]))
    if len(basin_id) == 7:
        basin_id = '0' + basin_id

    basin_id, em_model, xlb_mean, xub_mean, param_names, inpath_moasmo, numruns, iterend, suffix, num_objfunc, normdict, times = basin_id, em_model, xlb_mean, xub_mean, param_names, inpath_moasmo, numruns, iterend, suffix, num_objfunc, normdict, times

In [11]:
    basin_id = str(basin_id)

    # Check if the hru_id needs a leading zero (it should be 8 characters long)
    if len(basin_id) == 7:
        basin_id = '0' + basin_id
    
    # outpath = f'{inpath_moasmo}/param_sets_{suffix}/{basin_id}/'
    # os.makedirs(outpath, exist_ok=True)

    if num_objfunc == 1:
        ga_all_solutions, ga_all_outputs = run_ga_optimization(em_model, xlb_mean, xub_mean, num_runs=numruns, pop_size=100, num_generations=100, times=10)
        # outfile_ga = f'{outpath}/ga_output_iter{iterend}.npz'
        # if os.path.isfile(outfile_ga):
        #     dtmp = np.load(outfile_ga)
        #     ga_all_solutions = dtmp['ga_all_solutions']
        #     ga_all_outputs = dtmp['ga_all_outputs']
        # else:
        #     ga_all_solutions, ga_all_outputs = run_ga_optimization(em_model, xlb_mean, xub_mean, num_runs=numruns, pop_size=100, num_generations=100, times=times)
        #     np.savez_compressed(outfile_ga, ga_all_solutions=ga_all_solutions, ga_all_outputs=ga_all_outputs)
    
        final_solutions_array = np.array(ga_all_solutions)

    # elif num_objfunc == 2:
    #     outfile_nsga2 = f'{outpath}/nsga2_output_iter{iterend}.npz'
    #     if os.path.isfile(outfile_nsga2):
    #         dtmp = np.load(outfile_nsga2)
    #         nsga2_all_solutions = dtmp['nsga2_all_solutions']
    #         nsga2_all_outputs = dtmp['nsga2_all_outputs']
    #     else:
    #         nsga2_all_solutions, nsga2_all_outputs = run_nsga2_optimization(em_model, xlb_mean, xub_mean, num_runs=numruns, pop_size=100, num_generations=100)
    #         np.savez_compressed(outfile_nsga2, nsga2_all_solutions=nsga2_all_solutions, nsga2_all_outputs=nsga2_all_outputs)
    
    #     final_solutions_array = np.array(nsga2_all_solutions)
    
    df_info = pd.read_csv(f'{inpath_moasmo}/param_sets/{basin_id}/paramset_iter0_trial0.csv').loc[:13,:]
    df_info = df_info.loc[df_info['Value'] != 'None']
    
    df_info['Factor'] = np.nan
    df_info['Value'] = np.nan

    indexp = [np.where(param_names == p)[0][0] for p in df_info['Parameter'].values if p in param_names]

    # reverse normalization
    if 'method' in normdict:
        if normdict['method'] == 'z-score':
            print('reverse normalization')
            final_solutions_array = final_solutions_array*normdict['std'] + normdict['mean']


    # Read heightCanopyBottom from the file
    paramfile = f'/glade/campaign/cgd/tss/people/mozhgana/projects/SUMMA/settings/{basin_id}/trialParams.camels.nc'
    dataset = Dataset(paramfile)
    height_canopy_bottom_value = dataset.variables['heightCanopyBottom'][:][0]

    for i in range(final_solutions_array.shape[0]):
        outfile = f'{outpath}/paramset_iter{iterend}_trial{i}.csv'
        if os.path.isfile(outfile):
            continue

        dfi = df_info.copy()
        dfi['Value'] = final_solutions_array[i, indexp]

        # Ensure heightCanopyTop is greater than heightCanopyBottom
        if 'heightCanopyTop' in dfi['Parameter'].values:
            height_canopy_top_value = dfi[dfi['Parameter'] == 'heightCanopyTop']['Value'].values[0]

            if height_canopy_top_value <= height_canopy_bottom_value:
                # Adjust heightCanopyTop to be greater than heightCanopyBottom
                height_canopy_top_value = height_canopy_bottom_value + 0.5  # Add a buffer (e.g., 1.0)
                dfi.loc[dfi['Parameter'] == 'heightCanopyTop', 'Value'] = height_canopy_top_value
                print(f"Adjusted heightCanopyTop to {height_canopy_top_value} for basin {basin_id}")
        
        # dfi.to_csv(outfile, index=False)

    print('finish basin', basin_id)


Run GA 10 times and extract the best 1 solutions
reverse normalization
finish basin 01013500


In [32]:
basin_id

'01013500'

In [12]:
print(ga_all_outputs)
print(em_model.predict(xlb_mean[np.newaxis,:]))
print(em_model.predict(xub_mean[np.newaxis,:]))

[0.55142556]
[0.48702536]
[-0.2321801]


In [13]:
ga_all_solutions / xlb_mean

array([[ 0.9999883 ,  0.99972449,  0.99997361, -0.99854994,  0.99999612,
         0.94011305,  0.43063684,  0.9998742 ,  0.99970505,  0.99984253,
        -0.07839594,  0.89211014,  0.99999751,  0.99994814,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ]])

In [14]:
final_solutions_array[0] / xlb_mean_test[0]

/glade/derecho/scratch/guoqiang/tmp/ipykernel_30120/2064160181.py:1: RuntimeWarning: divide by zero encountered in divide
  final_solutions_array[0] / xlb_mean_test[0]
/glade/derecho/scratch/guoqiang/tmp/ipykernel_30120/2064160181.py:1: RuntimeWarning: invalid value encountered in divide
  final_solutions_array[0] / xlb_mean_test[0]


array([  1.04466429,   1.00013755,   1.00003962, 999.94575735,
         1.00003692,   3.99747905,   2.13702234,   1.01503781,
         1.00058995,   1.00088219,   1.01197509,   1.32202677,
         1.00002364,   1.00005375,   1.        ,   1.        ,
         1.        ,   1.        ,   1.        ,   1.        ,
         1.        ,   1.        ,   1.        ,   1.        ,
         1.        ,   1.        ,   1.        ,   1.        ,
         1.        ,   1.        ,   1.        ,   1.        ,
         1.        ,   1.        ,   1.        ,   1.        ,
         1.        ,   1.        ,          nan,   1.        ,
                nan,          nan,          nan,          nan,
               -inf,          nan,          nan,   1.        ,
                nan,          nan,          nan,          nan])

In [30]:
print(ga_all_outputs)
print(em_model.predict(xlb_mean[np.newaxis,:]))
print(em_model.predict(xub_mean[np.newaxis,:]))

[0.37369378]
[0.34949477]
[-0.12512727]


In [31]:
ga_all_solutions / xlb_mean

array([[ 0.99831824,  0.90915221,  0.63204818, -0.99095851,  0.99609719,
         0.94458203,  0.51997078,  0.94249149,  0.96941596,  0.94608948,
         0.6835617 ,  0.98975169,  0.97918253,  0.99982075,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ]])

In [49]:
final_solutions_array[0] / xlb_mean_test[0]

/glade/derecho/scratch/guoqiang/tmp/ipykernel_73482/2064160181.py:1: RuntimeWarning: divide by zero encountered in divide
  final_solutions_array[0] / xlb_mean_test[0]
/glade/derecho/scratch/guoqiang/tmp/ipykernel_73482/2064160181.py:1: RuntimeWarning: invalid value encountered in divide
  final_solutions_array[0] / xlb_mean_test[0]


array([  7.42185295,   1.04535723,   1.55239305, 996.15129491,
         1.03709946,   3.77379634,   1.95862182,   7.87453187,
         1.06117427,   1.30202385,   1.0035139 ,   1.03058889,
         1.19752615,   1.00018578,   1.        ,   1.        ,
         1.        ,   1.        ,   1.        ,   1.        ,
         1.        ,   1.        ,   1.        ,   1.        ,
         1.        ,   1.        ,   1.        ,   1.        ,
         1.        ,   1.        ,   1.        ,   1.        ,
         1.        ,   1.        ,   1.        ,   1.        ,
         1.        ,   1.        ,          nan,   1.        ,
                nan,          nan,          nan,          nan,
               -inf,          nan,          nan,   1.        ,
                nan,          nan,          nan,          nan])

In [47]:
df=pd.read_csv('/glade/campaign/cgd/tss/people/mozhgana/projects/SUMMA_Calib/param_sets_LSEspaceCV_1test/01013500/paramset_iter1_trial0.csv')
df['Value'].values / df['Lower'].values

array([6.30585882e+03, 1.82431461e+00, 1.04333292e+00, 1.70681383e+01,
       1.87172973e+00, 3.56465756e+00, 1.94973650e+00, 1.59393645e+01,
       1.08627072e+00, 1.16833937e+00, 1.00627789e+00, 1.00476224e+00,
       1.01632847e+00, 1.04614266e+00])

In [20]:
    df_att_use_train2 = df_att_use_train.drop(['hru_id'], axis=1)

    xlb_mean_train = np.nan * np.zeros([len(df_param_lb_train), len(inputnames)])
    xub_mean_train = np.nan * np.zeros([len(df_param_ub_train), len(inputnames)])   
    for i in range(len(df_param_lb_train)):
        param_lb_mean = df_param_lb_train.values[i, :]
        param_ub_mean = df_param_ub_train.values[i, :]
        attrvalues = df_att_use_train2.values[i,:]
        xlb_mean_train[i,:] = np.hstack([param_lb_mean, attrvalues])
        xub_mean_train[i,:] = np.hstack([param_ub_mean, attrvalues])

    xlb_mean_train_scaled = (xlb_mean_train - x_train_mean) / x_train_std
    xub_mean_train_scaled = (xub_mean_train - x_train_mean) / x_train_std
    normdict = {'method': 'z-score',
                'mean': x_train_mean,
                'std': x_train_std,
               }
    

In [21]:
em_model.predict(xlb_mean_train_scaled[[400],:])

array([0.09292251])

In [22]:
i=400
em_model.predict(x_all_scaled[i*400:(i+1)*400,:]).max()

np.float64(0.48507464279601153)